<a href="https://colab.research.google.com/github/mdjamina/m1_ml_lang_detector/blob/main/ML_lang_detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Corpus
[Tatoeba](https://tatoeba.org/fr/downloads
) est une collection de phrases et de traductions.

Extraction de l'archive téléchargée 

In [17]:
!unzip ./data/data.zip

Archive:  ./data/data.zip
warning [./data/data.zip]:  zipfile claims to be last disk of a multi-part archive;
  attempting to process anyway, assuming all parts have been concatenated
  together in order.  Expect "errors" and warnings...true multi-part support
  doesn't exist yet (coming soon).
file #1:  bad zipfile offset (local header sig):  4


## Chargement du corpus

In [18]:
import pandas as pd


data = pd.read_pickle("./data/data.pkl") 


data

,content,lang
0,我們試試看！,zho
1,我该去睡觉了。,zho
2,你在干什麼啊？,zho
3,這是什麼啊？,zho
4,今天是６月１８号，也是Muiriel的生日！,zho
...,...,...
10262652,Utálom a melóm!,hun
10262653,Ich habe den Ersatzschlüssel verloren.,deu
10262654,Mibe' fogadjunk?,hun
10262655,Utálok itt lenni!,hun


# pre-processing

## TODO

In [3]:
from sklearn.model_selection import train_test_split 

x_train, x_test, y_train, y_test = train_test_split(data['content'], data['lang'], test_size=0.33, random_state=1) 
# ramdom_state = meme corpus (meme decoupage, decouper en 2 train et test le corpus, car on a besoin de faire les tests)

In [ ]:
y_train.value_counts()

# Model

In [5]:
from joblib import Memory
from shutil import rmtree

# Create a temporary folder to store the transformers of the pipeline
mem = Memory(location="/tmp/cachedir", verbose=0)

In [6]:
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfVectorizer

model = Pipeline([('TfidfVectorizer', TfidfVectorizer()),
                     ('LinearSVC', LinearSVC()),],verbose=True,memory=mem)


model.fit(x_train, y_train)  




[Pipeline] ... (step 1 of 2) Processing TfidfVectorizer, total=  33.1s


/home/amina/.local/lib/python3.9/site-packages/sklearn/pipeline.py:348: UserWarning: Persisting input arguments took 26.34s to run.
If this happens often in your code, it can cause performance problems 
(results will be correct in all cases). 
The reason for this is probably some large input arguments for a wrapped
 function (e.g. large strings).
THIS IS A JOBLIB ISSUE. If you can, kindly provide the joblib's team with an
 example so that they can fix the problem.
  X, fitted_transformer = fit_transform_one_cached(


[Pipeline] ......... (step 2 of 2) Processing LinearSVC, total=11.6min


Pipeline(memory=Memory(location=cachedir/joblib),
         steps=[('TfidfVectorizer', TfidfVectorizer()),
                ('LinearSVC', LinearSVC())],
         verbose=True)

In [7]:
predictions = model.predict(x_test)

In [8]:
from sklearn import metrics
print(metrics.confusion_matrix(y_test,predictions))

[[ 13354      6      0 ...      0      0      0]
 [     3 180339      0 ...      0      0      0]
 [     0      0   7237 ...     40      0      0]
 ...
 [     0      1      4 ...  56558      0      0]
 [     0      0      0 ...      0   6906      0]
 [     0     17      0 ...      0      0   1412]]


In [9]:
#from sklearn import metrics
print(metrics.classification_report(y_test,predictions))

              precision    recall  f1-score   support

         ara       1.00      0.96      0.98     13846
         ber       0.91      0.93      0.92    194880
         bul       0.98      0.90      0.94      8016
         ces       0.97      0.97      0.97     20731
         dan       0.94      0.98      0.96     17825
         deu       1.00      1.00      1.00    191456
         ell       1.00      0.98      0.99     11611
         eng       1.00      1.00      1.00    518113
         est       0.98      0.85      0.91      1226
         fas       1.00      0.99      0.99      8191
         fin       1.00      0.98      0.99     45895
         fra       1.00      1.00      1.00    164772
         gle       0.99      0.94      0.97       861
         hbs       0.98      0.93      0.95     16937
         heb       1.00      0.99      1.00     65492
         hin       1.00      0.91      0.95      4706
         hun       1.00      0.98      0.99    117260
         hye       1.00    

In [10]:
metrics.accuracy_score(y_test,predictions)

0.9719847963262297

In [12]:
model.predict(['그러나 주로 개인적인 용도로 이용되는 위키도 있는데, 이를 개인 위키라고 한다. bonjour'])[0] 

'kor'

In [12]:
model.predict(['ויקי יכולה להיות שיטה טובה לשיתוף ידע בקהילות שפועלות באמצעות האינטרנט או בתוךיות. היא חלופה'])[0]

'heb'

In [13]:
model.predict(['einsetzt. Zudem nutzen auch viele '])[0]

'deu'

In [14]:
model.predict(['الكرام بتحويل الكتابة العربي الى كتابة مفهومة من قبل اغلب برامج التصميم مثل الفوتوشوب و الافترايفكتس و البريمير و الافد ميدا كومبوزر و السموك و برامج اخرى'])[0]

'ara'

In [15]:
model.predict(['से प्रत्येक एक विशिष्ट भाषा से संबंधित है। विकिपीडिया के अलावा, सार्वजनिक और निजी दोनों उपयोग में सैकड़ों हजारों अ'])[0]

'hin'

# Model persistence

In [14]:
from joblib import dump
dump(model, 'data/lang_detector.joblib') 

['data/lang_detector.joblib']